# Doğrusal Olmayan Regresyon Modelleri

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

# KNN

In [2]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[["League","Division","NewLeague"]])
y = df["Salary"]
X_ = df.drop(["Salary","League","Division","NewLeague"],axis = 1).astype("float64")
X = pd.concat([X_,dms[["League_N","Division_W","NewLeague_N"]]],axis = 1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 42)

In [3]:
knn_model = KNeighborsRegressor().fit(X_train,y_train)

In [4]:
knn_model

KNeighborsRegressor()

In [5]:
knn_model.n_neighbors

5

# Tahmin

In [6]:
y_pred = knn_model.predict(X_test)

In [7]:
np.sqrt(mean_squared_error(y_test,y_pred))

426.6570764525201

In [9]:
RMSE = []

for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train,y_train)
    y_pred = knn_model.predict(X_train)
    rmse = np.sqrt(mean_squared_error(y_train,y_pred))
    RMSE.append(rmse)
    print("k =",k,"İçin RMSE değeri:",rmse)
    

k = 1 İçin RMSE değeri: 0.0
k = 2 İçin RMSE değeri: 179.52761335480352
k = 3 İçin RMSE değeri: 205.20157172291863
k = 4 İçin RMSE değeri: 220.5139794876305
k = 5 İçin RMSE değeri: 239.6467132541376
k = 6 İçin RMSE değeri: 243.5904190007242
k = 7 İçin RMSE değeri: 258.1478781634636
k = 8 İçin RMSE değeri: 266.05374203349805
k = 9 İçin RMSE değeri: 269.73782093553376
k = 10 İçin RMSE değeri: 271.2798300436963


# Model Tuning

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
knn_params = {'n_neighbors': np.arange(1,30,1)}

In [20]:
knn = KNeighborsRegressor()

In [21]:
knn_cv_model = GridSearchCV(knn, knn_params, cv = 10)

In [22]:
knn_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

In [23]:
knn_cv_model.best_params_["n_neighbors"]

8

In [24]:
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"])

In [25]:
knn_tuned.fit(X_train,y_train)

KNeighborsRegressor(n_neighbors=8)

In [28]:
np.sqrt(mean_squared_error(y_test,knn_tuned.predict(X_test)))

413.7094731463598